This script loads behavioral mice data (from `biasedChoiceWorld` sessions and, separately, the last three sessions of training) only from mice that pass a given (stricter) training criterion defined in a *schema* uploaded by Anne Urai.
The data are slightly reformatted and saved as `.csv` files.

In [26]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference, data
from ibl_pipeline.analyses.behavior import PsychResults
from ibl_pipeline.utils import psychofit as psy
from ibl_pipeline.analyses import behavior as behavioral_analyses

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [30]:
# Get list of mice that satisfy given training criteria (from Anne Urai's analysis)

analyses = dj.create_virtual_module('analyses', 'group_shared_anneurai_analyses')

sess = ((acquisition.Session) *
        acquisition.Session.proj(session_date = 'DATE(session_start_time)'))*analyses.SessionTrainingStatus()*\
subject.Subject.proj('subject_nickname')  & 'task_protocol LIKE "%biased%"'

sess_dat = pd.DataFrame(sess.fetch(order_by='subject_nickname, session_start_time'))
mice_names = sess_dat['subject_nickname'].unique()
print(mice_names)

array(['CSHL_007', 'CSHL_010', 'CSK-les-008', 'DY_006', 'IBL-T4',
       'IBL_13', 'IBL_17', 'IBL_36', 'IBL_45', 'ibl_witten_06',
       'ibl_witten_07', 'KS003', 'KS004', 'NYU-01', 'ZM_1089', 'ZM_1092',
       'ZM_1093', 'ZM_1745', 'ZM_1746'], dtype=object)

In [31]:
sess_train  = (acquisition.Session & 'task_protocol LIKE "%training%"')
sess_stable = (acquisition.Session & 'task_protocol LIKE "%biased%"')
stable_mice_names = list()

# Perform at least this number of sessions
MinSessionNumber = 1 

def get_mouse_data(df):
    position_deg = 35.    # Stimuli appear at +/- 35 degrees

    # Create new dataframe
    datamat = pd.DataFrame()
    datamat['trial_num'] = df['trial_id']
    datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
    datamat['stim_probability_left'] = df['trial_stim_prob_left']
    signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
    datamat['contrast'] = np.abs(signed_contrast)
    datamat['position'] = np.sign(signed_contrast)*position_deg
    datamat['response_choice'] = df['trial_response_choice']
    datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
    datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
    datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
    datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
    datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

    # Since some trials have zero contrast, need to compute the alleged position separately
    datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
    datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
        datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)
    
    return datamat

In [33]:
# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Get mouse data for biased sessions
    behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
        * sess_stable.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id'))

    if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks            
        datamat = get_mouse_data(df)

        # Take mice that have performed a minimum number of sessions
        if np.max(datamat['session_num']) >= MinSessionNumber:

            # Should add 'N' to mice names that start with numbers?

            # Save dataframe to CSV file
            filename = mouse_nickname + '.csv'
            datamat.to_csv(filename,index=False)
            stable_mice_names.append(mouse_nickname)

            # Get mouse last sessions of training data
            behavior_train = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
                * sess_train.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
                * subject.SubjectLab.proj('lab_name')
            df_train = pd.DataFrame(behavior_train.fetch(order_by='subject_nickname, session_start_time, trial_id'))

            datamat_train = get_mouse_data(df_train)
            Nlast = np.max(datamat_train['session_num']) - 3
            datamat_final = datamat_train[datamat_train['session_num'] > Nlast]

            # Save final training dataframe to CSV file
            filename = mouse_nickname + '_endtrain.csv'
            datamat_final.to_csv(filename,index=False)
            

print(stable_mice_names)

['CSHL_007', 'CSHL_010', 'CSK-les-008', 'DY_006', 'IBL-T4', 'IBL_13', 'IBL_17', 'IBL_36', 'IBL_45', 'ibl_witten_06', 'ibl_witten_07', 'KS003', 'KS004', 'NYU-01', 'ZM_1089', 'ZM_1092', 'ZM_1093', 'ZM_1745', 'ZM_1746', 'CSHL_007', 'CSHL_010', 'CSK-les-008', 'DY_006', 'IBL-T4', 'IBL_13', 'IBL_17', 'IBL_36', 'IBL_45', 'ibl_witten_06', 'ibl_witten_07', 'KS003', 'KS004', 'NYU-01', 'ZM_1089', 'ZM_1092', 'ZM_1093', 'ZM_1745', 'ZM_1746']


In [35]:
sess_dat

,subject_uuid,session_start_time,session_uuid,session_number,session_end_time,session_lab,session_location,task_protocol,session_type,session_narrative,session_date,training_status,subject_nickname
0,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-20 15:59:32,9981029c-c3b5-4021-8651-1449a3210abc,1,2019-03-20 16:44:57,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-20,trained_1a,CSHL_007
1,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-21 16:43:59,0968d1cb-06f2-4c28-ba4a-f4d766be79d8,1,2019-03-21 17:30:13,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-21,trained_1a,CSHL_007
2,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-22 16:25:28,3449571e-282f-4e62-b084-9c9f5bd2b75b,2,2019-03-22 17:12:09,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-22,trained_1a,CSHL_007
3,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-25 17:35:43,c3d6caea-c64f-4d38-a1ee-f711b65f3b18,2,2019-03-25 18:16:20,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-25,trained_1a,CSHL_007
4,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-26 15:31:51,5108ed63-9d1c-4838-812a-0026c8a5a1ae,1,2019-03-26 15:48:04,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-26,trained_1a,CSHL_007
5,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-26 16:41:25,3cc89134-0d7c-4799-b1a9-2138508cb7eb,4,2019-03-26 17:26:26,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.7.6,Experiment,None,2019-03-26,trained_1a,CSHL_007
6,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-27 14:37:52,ca1fcefb-e8e2-4a60-a9ac-58fac8c06d9c,1,2019-03-27 15:27:24,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.8.1,Experiment,None,2019-03-27,trained_1a,CSHL_007
7,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-28 14:33:30,5595d4f5-2cd4-490a-8734-d28b3ecc07d5,2,2019-03-28 15:25:55,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.8.1,Experiment,None,2019-03-28,trained_1a,CSHL_007
8,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-29 17:26:32,90a4768b-c0ec-4dc3-8141-cad2889747e3,1,2019-03-29 18:06:42,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld3.8.1,Experiment,None,2019-03-29,trained_1a,CSHL_007
9,0936a724-2151-446f-98b2-92c3e9dd36dc,2019-04-01 14:14:58,4e2a1688-5efc-40bd-8e18-854111f87418,1,2019-04-01 14:49:57,churchlandlab,cshl-behavior-bpod1,_iblrig_tasks_biasedChoiceWorld4.0.1,Experiment,None,2019-04-01,trained_1a,CSHL_007
